In [104]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
import pandas as pd
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import functools
from datetime import date
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
password = getpass.getpass()

In [105]:


connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''select f.film_id, f.rental_duration, f.rental_rate, i.inventory_id, r.rental_id, r.rental_date, r.return_date, f.length, r.customer_id
from film f
join inventory i USING(film_id)
join rental r USING(inventory_id)
ORDER BY rental_date;'''
 
data = pd.read_sql_query(query, engine)
data.tail(20)


,film_id,rental_duration,rental_rate,inventory_id,rental_id,rental_date,return_date,length,customer_id
16025,270,7,0.99,1219,15293,2006-02-14 15:16:03,NaT,85,263
16026,273,6,0.99,1231,14361,2006-02-14 15:16:03,NaT,107,191
16027,281,6,0.99,1269,15778,2006-02-14 15:16:03,NaT,143,234
16028,286,7,2.99,1295,11757,2006-02-14 15:16:03,NaT,158,550
16029,300,5,4.99,1358,12130,2006-02-14 15:16:03,NaT,102,516
16030,301,4,0.99,1367,14760,2006-02-14 15:16:03,NaT,155,355
16031,301,4,0.99,1369,14734,2006-02-14 15:16:03,NaT,155,448
16032,314,3,0.99,1435,12792,2006-02-14 15:16:03,NaT,91,405
16033,326,6,2.99,1489,14243,2006-02-14 15:16:03,NaT,69,264
16034,330,7,4.99,1512,12897,2006-02-14 15:16:03,NaT,112,178


In [106]:
data = data[(data['rental_date'] < '2006-01-01')] # filtering for only data from 2005
data.tail()

,film_id,rental_duration,rental_rate,inventory_id,rental_id,rental_date,return_date,length,customer_id
15857,168,5,0.99,772,16045,2005-08-23 22:25:26,2005-08-25 23:54:26,67,14
15858,951,6,0.99,4364,16046,2005-08-23 22:26:47,2005-08-27 18:02:47,78,74
15859,452,4,0.99,2088,16047,2005-08-23 22:42:48,2005-08-25 02:48:48,122,114
15860,439,4,4.99,2019,16048,2005-08-23 22:43:07,2005-08-31 21:33:07,151,103
15861,585,4,0.99,2666,16049,2005-08-23 22:50:12,2005-08-30 01:01:12,105,393


In [107]:
data.tail()

,film_id,rental_duration,rental_rate,inventory_id,rental_id,rental_date,return_date,length,customer_id
15857,168,5,0.99,772,16045,2005-08-23 22:25:26,2005-08-25 23:54:26,67,14
15858,951,6,0.99,4364,16046,2005-08-23 22:26:47,2005-08-27 18:02:47,78,74
15859,452,4,0.99,2088,16047,2005-08-23 22:42:48,2005-08-25 02:48:48,122,114
15860,439,4,4.99,2019,16048,2005-08-23 22:43:07,2005-08-31 21:33:07,151,103
15861,585,4,0.99,2666,16049,2005-08-23 22:50:12,2005-08-30 01:01:12,105,393


In [108]:
def target_date(df):            
    if df == 5:
        return 'True'
    else:
        return 'False'

In [109]:
data['rental_month'] = pd.DatetimeIndex(data['rental_date']).month
data.head(20)

,film_id,rental_duration,rental_rate,inventory_id,rental_id,rental_date,return_date,length,customer_id,rental_month
0,80,7,2.99,367,1,2005-05-24 22:53:30,2005-05-26 22:04:30,148,130,5
1,333,7,2.99,1525,2,2005-05-24 22:54:33,2005-05-28 19:40:33,126,459,5
2,373,7,2.99,1711,3,2005-05-24 23:03:39,2005-06-01 22:12:39,156,408,5
3,535,6,0.99,2452,4,2005-05-24 23:04:41,2005-06-03 01:43:41,181,333,5
4,450,5,2.99,2079,5,2005-05-24 23:05:21,2005-06-02 04:33:21,84,222,5
5,613,5,0.99,2792,6,2005-05-24 23:08:07,2005-05-27 01:32:07,92,549,5
6,870,4,0.99,3995,7,2005-05-24 23:11:53,2005-05-29 20:34:53,123,269,5
7,510,6,4.99,2346,8,2005-05-24 23:31:46,2005-05-27 23:33:46,181,239,5
8,565,6,4.99,2580,9,2005-05-25 00:00:40,2005-05-28 00:22:40,56,126,5
9,396,5,4.99,1824,10,2005-05-25 00:02:21,2005-05-31 22:44:21,62,399,5


In [110]:
data['target'] = data['rental_month'].apply(target_date) 
data.head()

,film_id,rental_duration,rental_rate,inventory_id,rental_id,rental_date,return_date,length,customer_id,rental_month,target
0,80,7,2.99,367,1,2005-05-24 22:53:30,2005-05-26 22:04:30,148,130,5,True
1,333,7,2.99,1525,2,2005-05-24 22:54:33,2005-05-28 19:40:33,126,459,5,True
2,373,7,2.99,1711,3,2005-05-24 23:03:39,2005-06-01 22:12:39,156,408,5,True
3,535,6,0.99,2452,4,2005-05-24 23:04:41,2005-06-03 01:43:41,181,333,5,True
4,450,5,2.99,2079,5,2005-05-24 23:05:21,2005-06-02 04:33:21,84,222,5,True


In [111]:
data.drop(['rental_date','return_date'],axis=1, inplace=True)
data.head()

,film_id,rental_duration,rental_rate,inventory_id,rental_id,length,customer_id,rental_month,target
0,80,7,2.99,367,1,148,130,5,True
1,333,7,2.99,1525,2,126,459,5,True
2,373,7,2.99,1711,3,156,408,5,True
3,535,6,0.99,2452,4,181,333,5,True
4,450,5,2.99,2079,5,84,222,5,True


In [112]:
data_num = data._get_numeric_data()
data_num.head()

,film_id,rental_duration,rental_rate,inventory_id,rental_id,length,customer_id,rental_month
0,80,7,2.99,367,1,148,130,5
1,333,7,2.99,1525,2,126,459,5
2,373,7,2.99,1711,3,156,408,5
3,535,6,0.99,2452,4,181,333,5
4,450,5,2.99,2079,5,84,222,5


In [113]:
transformer = MinMaxScaler().fit(data_num)
X_num_normalized = transformer.transform(data_num)
X_num_normalized = pd.DataFrame(X_num_normalized, columns=data_num.columns)
X_num_normalized.tail()

,film_id,rental_duration,rental_rate,inventory_id,rental_id,length,customer_id,rental_month
15857,0.167167,0.50,0.0,0.168341,0.999751,0.151079,0.021739,1.0
15858,0.950951,0.75,0.0,0.952620,0.999813,0.230216,0.122074,1.0
15859,0.451451,0.25,0.0,0.455677,0.999875,0.546763,0.188963,1.0
15860,0.438438,0.25,1.0,0.440611,0.999938,0.755396,0.170569,1.0
15861,0.584585,0.25,0.0,0.581878,1.000000,0.424460,0.655518,1.0


In [114]:
data_cat = data[['target']]
data_cat.head()

,target
0,True
1,True
2,True
3,True
4,True


In [115]:
encoder = OneHotEncoder(drop='first').fit(data_cat)
encoded = encoder.transform(data_cat).toarray()
cols = encoder.get_feature_names(input_features=data_cat.columns)
onehot_encoded = pd.DataFrame(encoded, columns=['target'])
onehot_encoded.shape
onehot_encoded.head(10)

,target
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0
6,1.0
7,1.0
8,1.0
9,1.0


In [116]:
data_transformed = pd.concat([X_num_normalized, onehot_encoded], axis=1)
data_transformed.head()
    

,film_id,rental_duration,rental_rate,inventory_id,rental_id,length,customer_id,rental_month,target
0,0.079079,1.00,0.5,0.079913,0.000000,0.733813,0.215719,0.0,1.0
1,0.332332,1.00,0.5,0.332751,0.000062,0.575540,0.765886,0.0,1.0
2,0.372372,1.00,0.5,0.373362,0.000125,0.791367,0.680602,0.0,1.0
3,0.534535,0.75,0.0,0.535153,0.000187,0.971223,0.555184,0.0,1.0
4,0.449449,0.50,0.5,0.453712,0.000249,0.273381,0.369565,0.0,1.0


In [134]:
y = data_transformed['target']
X = data_transformed.drop(['target','rental_month'], axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [135]:
X.head()

,film_id,rental_duration,rental_rate,inventory_id,rental_id,length,customer_id
0,0.079079,1.00,0.5,0.079913,0.000000,0.733813,0.215719
1,0.332332,1.00,0.5,0.332751,0.000062,0.575540,0.765886
2,0.372372,1.00,0.5,0.373362,0.000125,0.791367,0.680602
3,0.534535,0.75,0.0,0.535153,0.000187,0.971223,0.555184
4,0.449449,0.50,0.5,0.453712,0.000249,0.273381,0.369565


In [136]:
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='multinomial').fit(X_train, y_train)

In [137]:
classification2 = LogisticRegression(random_state=0, solver='saga',
                  multi_class='multinomial').fit(X_train, y_train)

In [141]:
predictions = classification.predict(X_test)
classification.score(X_test, y_test)

0.9965332492908919